In [1]:
import pandas as pd
import plotly.graph_objs as go

In [2]:
# Source: Global Monitoring Laboratory, NOAA
co2_global_url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.txt"
co2_daily_url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_trend_gl.txt"

In [3]:
# Read the Global Montly Mean CO2 txt
co2_global = pd.read_csv(co2_global_url, 
                         sep=r"\s+", 
                         comment='#', 
                         names=["year",
                                "month", 
                                "decimal", 
                                "average", 
                                "trend"])

In [4]:
# Plot the Global Montly Mean CO2
fig = go.Figure([go.Scatter(name="Global Montly Mean CO2", 
                            x=co2_global["decimal"], 
                            y=co2_global["average"], 
                            line=dict(color="crimson"),
                            hovertemplate='GMM CO<sub>2</sub>: %{y}<extra></extra>',
                            showlegend=False),
                 go.Scatter(name="Global Trend CO2", 
                            x=co2_global["decimal"], 
                            y=co2_global["trend"], 
                            line=dict(color="black"),
                            hovertemplate='GT CO<sub>2</sub>: %{y}<extra></extra>',
                            showlegend=False)])
fig.layout.title.text = "Global Montly Mean CO<sub>2</sub> (1980-2021)"
fig.update_layout(xaxis_title="Years",
                  yaxis_title="ppm")
fig.show()

In [5]:
# Read the Global daily CO2 txt
co2_daily = pd.read_csv(co2_daily_url, 
                        sep=r"\s+", 
                        comment='#', 
                        names=["year", 
                               "month", 
                               "day", 
                               "cycle", 
                               "trend"])

In [6]:
# Create a Datetime column from year, month, day
co2_daily["date"] = pd.to_datetime(co2_daily[['year', 
                                              'month', 
                                              'day']])

co2_daily["date_str"] = co2_daily.apply(lambda r: "{}, {} {}".format(r["date"].strftime("%A"), 
                       r["date"].strftime("%B"), 
                       r["date"].day), axis=1)

In [7]:
# Get the most recent CO2 Global trend value
last_value_co2 = co2_daily.iloc[-1]

In [8]:
fig = go.Figure([
    go.Indicator(mode = "number",
                value = last_value_co2["trend"],
                title = {"text": last_value_co2["date_str"]},
                number = {'valueformat':'.2f'},
                domain = {'y': [0, 1], 'x': [0.25, 0.75]}),
                 
    go.Scatter(name="Global Seasonal Cycle", 
                x=co2_daily["date"], 
                y=co2_daily["cycle"], 
                line=dict(color="crimson"),
                hovertemplate='GSC CO<sub>2</sub>: %{y}<extra></extra>',
                showlegend=False),
    
    go.Scatter(name="Global Trend", 
                x=co2_daily["date"], 
                y=co2_daily["trend"], 
                line=dict(color="black"),
                hovertemplate='GT CO<sub>2</sub>: %{y}<extra></extra>',
                showlegend=False)])

fig.layout.title.text = "Daily Global CO<sub>2</sub> (2011-2021)"
fig.update_layout(xaxis_title="Years",
                  yaxis_title="ppm")
fig.show()